In [ ]:
import requests
import tarfile
import torch
from IPython.display import Image
import json
import torchvision.models as models
import torchvision.models.segmentation.lraspp
import torch
!git clone https://github.com/pytorch/vision

Cloning into 'vision'...
remote: Enumerating objects: 470584, done.
remote: Counting objects: 100% (51042/51042), done.
remote: Compressing objects: 100% (2377/2377), done.
remote: Total 470584 (delta 48741), reused 50832 (delta 48588), pack-reused 419542
Receiving objects: 100% (470584/470584), 924.23 MiB | 22.47 MiB/s, done.
Resolving deltas: 100% (437844/437844), done.


In [ ]:
!sudo apt install swig
!git clone https://github.com/yvan674/obb_anns
!python3 setup.py develop

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 3s (434 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
de

In [ ]:
!du -sh /content/ds2_dense/deepscores_train.json

In [ ]:
r = requests.get('https://zenodo.org/records/4012193/files/ds2_dense.tar.gz?download=1')

with open('ds2_dense.tar.gz', 'wb') as f:
    f.write(r.content)
with tarfile.open('ds2_dense.tar.gz', 'r:gz') as tar:
    tar.extractall()

In [ ]:
data = []
with open('/content/ds2_dense/deepscores_train.json', 'r') as file:
    data = json.load(file)

In [ ]:

print(len(list(data['categories'].keys())))


model = torchvision.models.segmentation.lraspp.LRASPP(backbone=torchvision.models.mobilenetv3.MobileNetV3,
                                                      low_channels=10, high_channels=10, num_classes=len(list(data['categories'].keys())))

!torchrun --nproc_per_node=8 vision/references/segmentation/train.py --dataset coco -b 4 --model lraspp_mobilenet_v3_large --wd 0.000001 --weights-backbone MobileNet_V3_Large_Weights.IMAGENET1K_V1


208
[2024-03-02 17:36:52,007] torch.distributed.run: [WARNING] 
[2024-03-02 17:36:52,007] torch.distributed.run: [WARNING] *****************************************
[2024-03-02 17:36:52,007] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-03-02 17:36:52,007] torch.distributed.run: [WARNING] *****************************************
[2024-03-02 17:37:28,643] torch.distributed.elastic.agent.server.api: [WARNING] Received Signals.SIGINT death signal, shutting down workers
[2024-03-02 17:37:28,644] torch.distributed.elastic.multiprocessing.api: [WARNING] Sending process 25217 closing signal SIGINT
[2024-03-02 17:37:28,644] torch.distributed.elastic.multiprocessing.api: [WARNING] Sending process 25218 closing signal SIGINT
[2024-03-02 17:37:28,644] torch.distributed.elastic.multiprocessing.api

In [ ]:
import torch
import torchvision
from torchvision.models.detection import MaskRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import DeepScoresV2
from engine import train_one_epoch, evaluate

# Define transforms for data augmentation and normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    # Add more transforms as needed
])

# Load DeepScoresV2 dataset
dataset_train = DeepScoresV2(root='./data', train=True, transform=transform)
dataset_test = DeepScoresV2(root='./data', train=False, transform=transform)

# Define data loaders
data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, num_workers=4)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=4)

# Load pre-trained Mask R-CNN model
model = MaskRCNN(
    backbone=torchvision.models.resnet50(pretrained=True).backbone,
    num_classes=91  # 90 classes + background
)

# Define optimizer and learning rate scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Move model to device (CPU or GPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=10)
    lr_scheduler.step()

    # Evaluation
    evaluate(model, data_loader_test, device=device)

# Save the trained model
torch.save(model.state_dict(), 'mask_rcnn_deepscoresv2.pth')
